# Polynomiale Nichtlineare Regression

In [2]:
import pandas as pd
df = pd.read_csv('trip_weather_hourly.csv')
df
date_column = df['start_date']
from datetime import datetime, timedelta
date_colum1 = [datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in date_column]
# aus den Zeilen die Datumselemente auslesen
start_date_month = pd.DataFrame([i.month for i in date_colum1], columns = ['month'])
start_date_weekday = pd.DataFrame([i.weekday() for i in date_colum1], columns = ['weekday'])
start_date_hour = pd.DataFrame([i.hour for i in date_colum1], columns = ['hour'])
df_expand = pd.concat([start_date_month, start_date_weekday, start_date_hour, df], axis = 1)
df_expand

,month,weekday,hour,start_date,number_trips,duration,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8,3,9,2013-08-29 09:00:00,12,1022.500000,74.0,68.0,61.0,61.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,8,3,10,2013-08-29 10:00:00,14,1677.857143,74.0,68.0,61.0,61.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
2,8,3,11,2013-08-29 11:00:00,42,2203.809524,74.0,68.0,61.0,61.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
3,8,3,12,2013-08-29 12:00:00,120,934.833333,74.0,68.0,61.0,61.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
4,8,3,13,2013-08-29 13:00:00,87,3934.103448,74.0,68.0,61.0,61.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17578,8,0,19,2015-08-31 19:00:00,72,496.652778,78.0,69.0,60.0,58.0,...,10.0,9.0,18.0,9.0,21.0,0,1.0,NaN,246.0,94107
17579,8,0,20,2015-08-31 20:00:00,35,544.514286,78.0,69.0,60.0,58.0,...,10.0,9.0,18.0,9.0,21.0,0,1.0,NaN,246.0,94107
17580,8,0,21,2015-08-31 21:00:00,13,523.230769,78.0,69.0,60.0,58.0,...,10.0,9.0,18.0,9.0,21.0,0,1.0,NaN,246.0,94107
17581,8,0,22,2015-08-31 22:00:00,2,190.500000,78.0,69.0,60.0,58.0,...,10.0,9.0,18.0,9.0,21.0,0,1.0,NaN,246.0,94107


#### TrainTestSplit

In [4]:
from sklearn.model_selection import train_test_split
X = df_expand[['month','weekday','hour']]
y = df_expand['number_trips']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14066, 3), (3517, 3), (14066,), (3517,))

#### 2. One Hot Encoding

In [8]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit(X)
X_train_ohe = ohe.transform(X_train)
X_test_ohe = ohe.transform(X_test)

#### Polynomiales Modell aufstellen
# y = c1*x1+c2*x2+c3*x3...+c11*x1^2+c22*x2^2+c33*x3^2...+c12*x1*x2+c13*x1*x3+...

In [9]:
from sklearn.preprocessing import PolynomialFeatures
poly= PolynomialFeatures(degree = 2, include_bias = False)
poly.fit(X_train_ohe)
X_train_poly = poly.transform(X_train_ohe)
X_test_poly = poly.transform(X_test_ohe)
X_train_poly.shape

(14066, 989)

#### Regression vornehmen

In [22]:
from sklearn.linear_model import LinearRegression
linear_poly = LinearRegression()
linear_poly.fit(X_train_poly, y_train)
r2 = linear_poly.score(X_test_poly, y_test)
print('r2 = '+str(r2))
y_pred = linear_poly.predict(X_test_poly)
ergebnis = pd.DataFrame()
import numpy as np
np.absolute((y_test-y_pred)/y_test)



r2 = 0.8615023115106883


2449     0.193864
2337     0.198953
5567     0.140811
9905          inf
11499    0.808530
           ...   
8475     0.227681
2888     1.491147
9160          inf
14906    0.033354
10466    0.045136
Name: number_trips, Length: 3517, dtype: float64